In [25]:
import imageio
import torch

img_arr = imageio.imread('data/bobby.jpg')

img_arr.shape

# img_array is a numpy array like object with three dimensions: two spatial dimensions, width and height and a third dimension corresponding to the
# red, green and blue channels. 

(720, 1280, 3)

In [26]:
# use the tensors 'permute' method with the old dimensions for each new dimension to get an appropiate layout

# given an input tensor h * w * c we get a proper layout by having channel 2 first and then channels 0 and 1

img = torch.from_numpy(img_arr)
out = img.permute(2,0,1)

# important to note that this operation does not make a copy of the tensor data

In [27]:
# to create a dataset of multiple images to use as an input for neural networks, store the images in a batch along the first dimension
# to obtain a n*c*h*w tensor.

# create a tensor of an appropiate size and fill it with images loaded from a directory

batch_size = 3 
batch = torch.zeros(batch_size, 3, 256, 256, dtype=torch.uint8)

# this indicates our batch will consist of three RGB images 256 pixels in height and 256 pixels in length 

# each colour will be represented as an 8 bit integer

In [28]:
import os 
data_dir = 'data/cats/'

filenames = [name for name in os.listdir(data_dir) if os.path.splitext(name)[-1] =='.png']

for i, filename in enumerate(filenames):
    img_arr = imageio.imread(os.path.join(data_dir,filename))
    img_t = torch.from_numpy(img_arr)
    img_t = img_t.permute(2,0,1)
    img_t = img_t[:3]   # only keep the first 3 channels, some images have an alpha channel indicating transparency, but this network only wants RGB
    batch[i] = img_t
    

NORMALIZING THE DATA:
neural networks work best with data represented as floating point tensors with a range of [0,1] or [-1,1].

In [29]:
# the easiest way is to divide the values of the pixels by 255 (max. representable number in 8 bit unsigned)
batch = batch.float()
batch /= 255.0

In [30]:
# can normalize the data to mean = 0 and sd = 1
n_channels = batch.shape[1]
for c in range(n_channels):
    mean = torch.mean(batch[:,c])
    std = torch.std(batch[:,c])
    batch[:,c] = (batch[:,c] - mean) / std

3D IMAGES: VOLUMETRIC DATA
CT scans have only a single intensity channel, similar to grayscale images. This means that the channel dimension is left out in native data formats, typically the raw data has three dimensions.

By stacking individual 2d slices into a 3d tensor we can build volumetric data representing the 3d anatomy.

we have a 5d tensor of n * c * d * h * w with d = depth

In [31]:
import imageio
dir_path = 'data/ds/ls'
vol_arr = imageio.volread(dir_path, 'DICOM')
vol_arr.shape

Reading DICOM (examining files): 1/99 files (1.0%99/99 files (100.0%)
  Found 1 correct series.
Reading DICOM (loading data): 85/99  (85.999/99  (100.0%)


(99, 512, 512)

In [32]:
# the layout is different to what pytorch expects, due to have no channel information.
# we will have to make room for the channel dimension using unsqueeze.
vol = torch.from_numpy(vol_arr).float()
vol = torch.unsqueeze(vol,0)
vol.shape

torch.Size([1, 99, 512, 512])

LOADING A CSV FILE
can use pandas but will use numpy

In [33]:
import csv, numpy as np
wine_path = 'data/tab_wine/wine_equality.csv'
wineq_numpy = np.loadtxt(wine_path, dtype=np.float32,delimiter=';', skiprows =1)

# here we have prescribed the type of the 2d array (32 bit float), the delimiter and that the first column should not be read since it 
# contains the column names

In [34]:
col_list = next(csv.reader(open(wine_path), delimiter=';'))
wineq_numpy.shape, col_list

((4898, 12),
 ['fixed acidity',
  'volatile acidity',
  'citric acid',
  'residual sugar',
  'chlorides',
  'free sulfur dioxide',
  'total sulfur dioxide',
  'density',
  'pH',
  'sulphates',
  'alcohol',
  'quality'])

In [35]:
# convert the numpy array to a pytorch tensor
wineq = torch.from_numpy(wineq_numpy)
wineq.shape,wineq.dtype

(torch.Size([4898, 12]), torch.float32)

In [37]:
# could treat quality as continouos variable and perform regression or use it as label and perform classification
# however in both cases remove the column from the tensor of input data and store in a separate tensor

data = wineq[:, :-1]  # select all rows and all columns bar last
data, data.shape

target= wineq[:, -1]   # select all rows and last column
target, target.shape

(tensor([6., 6., 6.,  ..., 6., 7., 6.]), torch.Size([4898]))

In [38]:
# can simply transform the the target tensor to a tensor of labels

target = wineq[:, -1].long()
target

tensor([6, 6, 6,  ..., 6, 7, 6])

In [39]:
# one hot coding the target tensor
target_onehot = torch.zeros(target.shape[0], 10)
target_onehot.scatter_(1, target.unsqueeze(1), 1.0) 

# any method (squeeze_ above) with an underscore modifys the tensor in place.

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

The arguments for scatter_ are :
    1. the dimension along which the following two arguments are specified
    2. a column tensor indicating the indices of the elements to scatter
    3. a tensor containing the elements to scatter or a single scalar to scatter (1, in this case)

The second argument of scatter_ (index tensor) is required to have the same dimensions as the tensor we scatter in to. Since target_onehot as two dimensions (4898 * 10), we need to add an extra dummy dimension to target using unsqueeze

In [40]:
target_unsqueezed = target.unsqueeze(1)
target_unsqueezed

tensor([[6],
        [6],
        [6],
        ...,
        [6],
        [7],
        [6]])

In [45]:
# obtain the mean and sd
data_mean = torch.mean(data, dim=0) # dim = 0 indicates the reduction in performed on dimension 0
data_mean

data_var = torch.var(data, dim=0)
data_var

11

In [46]:
# now we can normalize the data
data_normalized = (data - data_mean) / torch.sqrt(data_var)
data_normalized

tensor([[ 1.7208e-01, -8.1761e-02,  2.1326e-01,  ..., -1.2468e+00,
         -3.4915e-01, -1.3930e+00],
        [-6.5743e-01,  2.1587e-01,  4.7996e-02,  ...,  7.3995e-01,
          1.3422e-03, -8.2419e-01],
        [ 1.4756e+00,  1.7450e-02,  5.4378e-01,  ...,  4.7505e-01,
         -4.3677e-01, -3.3663e-01],
        ...,
        [-4.2043e-01, -3.7940e-01, -1.1915e+00,  ..., -1.3130e+00,
         -2.6153e-01, -9.0545e-01],
        [-1.6054e+00,  1.1666e-01, -2.8253e-01,  ...,  1.0049e+00,
         -9.6251e-01,  1.8574e+00],
        [-1.0129e+00, -6.7703e-01,  3.7852e-01,  ...,  4.7505e-01,
         -1.4882e+00,  1.0448e+00]])

FINDING THRESHOLDS
determine which rows in target correspond to <= 3

In [47]:
bad_indexes = target <= 3
bad_indexes.shape, bad_indexes.dtype, bad_indexes.sum()

# returned a tensor of boolean values, 20 of which are true

(torch.Size([4898]), torch.bool, tensor(20))

In [49]:
# can now use bad_indexes to index the data tensor, this will filter the tensor to only return rows corresponding to True
bad_data = data[bad_indexes]
bad_data.shape

# the tensor has 20 rows and 11 columns

torch.Size([20, 11])

In [50]:
# group the wines into good, mid and bad. take the mean of each column

bad_data = data[target <= 3]
mid_data = data[(target > 3) & (target < 7)]
good_data = data[target >= 7]

bad_mean = torch.mean(bad_data, dim = 0)
mid_mean = torch.mean(mid_data, dim=0)
good_mean = torch.mean(good_data, dim =0)

for i, args in enumerate(zip(col_list, bad_mean, mid_mean, good_mean)):
    print('{:2} {:20} {:6.2f} {:6.2f} {:6.2f}'.format(i,*args))

 0 fixed acidity          7.60   6.89   6.73
 1 volatile acidity       0.33   0.28   0.27
 2 citric acid            0.34   0.34   0.33
 3 residual sugar         6.39   6.71   5.26
 4 chlorides              0.05   0.05   0.04
 5 free sulfur dioxide   53.33  35.42  34.55
 6 total sulfur dioxide 170.60 141.83 125.25
 7 density                0.99   0.99   0.99
 8 pH                     3.19   3.18   3.22
 9 sulphates              0.47   0.49   0.50
10 alcohol               10.34  10.26  11.42
